In [ ]:
import chart_studio.plotly as py
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
np.random.seed(42)

## Building our robotic arm (Construindo nosso manipulador)

We will start our robotic arm in the standard position as we can see below:

----------
Para a construção do nosso manipulador, inicializaremos ele na posição padrão.

In [ ]:
from src.robot import Manipulator, Joint

joint1 = Joint(100, 0, 90, 90)
joint2 = Joint(0, 50, 0, 0, link=joint1)
joint3 = Joint(0, 50, 0, 0, link=joint2)

robot = Manipulator([joint1, joint2, joint3])

In [ ]:
robot.forward_kinematics()
robot.static_plot()

In [ ]:
print('Final Matrix:\n', np.round(joint3.updated_matrix))

In [ ]:
robot.get_X(), robot.get_Y(), robot.get_Z()

In [ ]:
robot.update_theta(0, 0)
robot.update_theta(1, -90)
robot.update_theta(2, -10)


robot.forward_kinematics()
robot.static_plot()

In [ ]:
robot.get_X(), robot.get_Y(), robot.get_Z()

## Generating our dataset (Gerando nossa base de dados)

To generate our dataset we need to pass a list of $\theta$ to our robotic arm and save the endeffector's coordinates. So, during the training, our input will be the endeffector's coordinates and our output will be $\theta$ values.

-------

Para gerar nossos dados de treino, passaremos uma série de valores de $\theta$ para nosso manipulador e salvaremos os valores das coordenadas do endeffector. Assim, na hora de treinar, nosso input será as coordenadas do endeffector e nosso output os valores de theta.

In [ ]:
from src.data.make_dataset import make_dataset

In [ ]:
theta1_zone1 = list(range(0, 100, 10)) + list(range(180, 280, 10))
theta1_zone2 = list(range(90, 190, 10)) + list(range(270, 370, 10))
theta2 = list(range(-90, 100, 10))
theta3 = list(range(-90, 100, 10))

In [ ]:
X_zone1, y_zone1 = make_dataset(robot, [theta1_zone1, theta2, theta3], 'data_zone1')
X_zone2, y_zone2 = make_dataset(robot, [theta1_zone2, theta2, theta3], 'data_zone2')

In [ ]:
data_for_dataframe = []
for x, y in zip(X_zone1, y_zone1):
    x_ = list(x)
    x_.extend(list(y))
    x_.extend([1])
    data_for_dataframe.append(x_)
for x, y in zip(X_zone2, y_zone2):
    x_ = list(x)
    x_.extend(list(y))
    x_.extend([2])
    data_for_dataframe.append(x_)

In [ ]:
df = pd.DataFrame(data_for_dataframe, columns=['X', 'Y', 'Z', 't1', 't2', 't3', 'zona'])
print('Shape inicial: ', df.shape)

for column in ['X', 'Y', 'Z']:
    df[column] = df[column].apply(lambda x: round(x, 4))

df_clean = df.drop_duplicates(subset=['X','Y','Z', 'zona'], keep='first')
print('Shape final: ', df_clean.shape)

In [ ]:
df_clean['zona'].value_counts()

In [ ]:
df_clean.head()

In [ ]:
df_zona1 = df_clean[df_clean.zona == 1]
df_zona2 = df_clean[df_clean.zona == 2]
df_total = df_clean.drop_duplicates(subset=['X','Y','Z'], keep='first')

In [ ]:
X_zone1, y_zone1 = df_zona1[['X', 'Y', 'Z']].values, df_zona1[['t1', 't2', 't3']].values
X_zone2, y_zone2 = df_zona2[['X', 'Y', 'Z']].values, df_zona2[['t1', 't2', 't3']].values
X_total, y_total = df_clean[['X', 'Y', 'Z']].values, df_clean[['t1', 't2', 't3']].values

In [ ]:
X_zone2[324], y_zone2[324]

### Visualização da área de trabalho gerada

In [ ]:
theta1 = list(range(0, 360, 10))

In [ ]:
robot.plot_workspace([theta1, theta2, theta3], [90, 0, 0])

## Visualização das duas zonas de dados

In [ ]:
initial = [90, 0, 0]
theta1_zone1 = list(range(0, 100, 10))
theta1_zone2 = list(range(180, 280, 10))
theta2 = list(range(-90, 100, 10))
theta3 = list(range(-90, 100, 10))
X, Y, Z = [], [], []

for t1 in theta1_zone1:
    for t2 in theta2:
        for t3 in theta3:
            robot.update_theta(0, t1)
            robot.update_theta(1, t2)
            robot.update_theta(2, t3)
            
            robot.forward_kinematics()
            if robot.get_X() >= 0 and robot.get_Y()>=0:
                X.append(robot.get_X())
                Y.append(robot.get_Y())
                Z.append(robot.get_Z())

for i in range(len(robot.joints)):
    robot.update_theta(i, initial[i])
robot.forward_kinematics()
X_ = robot.get_all_X()
Y_ = robot.get_all_Y()
Z_ = robot.get_all_Z()

fig = go.Figure(data=go.Mesh3d(x=X, y=Y, z=Z,
                   alphahull=5,
                   opacity=0.5,
                   color='orange'
    ))

X, Y, Z = [], [], []

for t1 in theta1_zone2:
    for t2 in theta2:
        for t3 in theta3:
            robot.update_theta(0, t1)
            robot.update_theta(1, t2)
            robot.update_theta(2, t3)

            robot.forward_kinematics()
            if round(robot.get_X(),4) <= 0 and round(robot.get_Y(),4) <= 0:
                X.append(robot.get_X())
                Y.append(robot.get_Y())
                Z.append(robot.get_Z())

fig.add_trace(go.Mesh3d(x=X, y=Y, z=Z,
                   alphahull=5,
                   opacity=0.5,
                   color='orange'
    ))

initial = [90, 0, 0]
theta1_zone1 = list(range(90, 190, 10))
theta1_zone2 = list(range(270, 370, 10))

X, Y, Z = [], [], []
X_Y = []
    
for t1 in theta1_zone1:
    for t2 in theta2:
        for t3 in theta3:
            robot.update_theta(0, t1)
            robot.update_theta(1, t2)
            robot.update_theta(2, t3)

            robot.forward_kinematics()
            if round(robot.get_Y(),4) >= 0 and round(robot.get_X(),4) <= 0:
                X.append(robot.get_X())
                Y.append(robot.get_Y())
                Z.append(robot.get_Z())

for i in range(len(robot.joints)):
    robot.update_theta(i, initial[i])
robot.forward_kinematics()
X_ = robot.get_all_X()
Y_ = robot.get_all_Y()
Z_ = robot.get_all_Z()

fig.add_trace(go.Mesh3d(x=X, y=Y, z=Z,
                   alphahull=5,
                   opacity=0.6,
                   color='blue'
    ))

X, Y, Z = [], [], []
for t1 in theta1_zone2:
    for t2 in theta2:
        for t3 in theta3:
            robot.update_theta(0, t1)
            robot.update_theta(1, t2)
            robot.update_theta(2, t3)

            robot.forward_kinematics()
            if round(robot.get_Y(), 4) <= 0 and round(robot.get_X(), 4) >= 0:
                X.append(robot.get_X())
                Y.append(robot.get_Y())
                Z.append(robot.get_Z())

fig.add_trace(go.Mesh3d(x=X, y=Y, z=Z,
                   alphahull=5,
                   opacity=0.6,
                   color='blue'
    ))

fig.add_trace(go.Scatter3d(
                    x=X_, y=Y_, z=Z_,
                    marker=dict(
                        size=0,
                        color=3,
                        colorscale='Viridis',
                    ),
                    line=dict(
                        color='darkblue',
                        width=5
                    )
                ))
fig.update_layout(
                   width=700, height=700, autosize=True, scene_aspectmode='cube', showlegend=False)
fig.show()

## Treinamento da rede neural

### Pré-processamento dos dados

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
indices = np.arange(X_zone1.shape[0])
np.random.shuffle(indices)

X_zone1 = X_zone1[indices]
y_zone1 = y_zone1[indices]

indices = np.arange(X_zone2.shape[0])
np.random.shuffle(indices)

X_zone2 = X_zone2[indices]
y_zone2 = y_zone2[indices]

indices = np.arange(X_total.shape[0])
np.random.shuffle(indices)

X_total = X_total[indices]
y_total = y_total[indices]

In [ ]:
scaler_X1 = StandardScaler()
X_zone1 = scaler_X1.fit_transform(X_zone1)

scaler_X2 = StandardScaler()
X_zone2 = scaler_X2.fit_transform(X_zone2)

scaler_Xtotal = StandardScaler()
X_total = scaler_Xtotal.fit_transform(X_total)

## Trejetória de teste

In [ ]:
test_set_input1 = []
test_set_output1 = []

test_set_input2 = []
test_set_output2 = []

In [ ]:
X, Y, Z = [], [], []
input_pred = []
final_X, final_Y, final_Z = [], [], []
points = []
robot.update_theta(0, 0)
robot.update_theta(1, 0)
robot.update_theta(2, -90)
theta2 = 0
theta3 = -90
for k in range(3):
    for i in range(0, 360, 15):
        robot.update_theta(0, i)
        robot.update_theta(1, theta2)
        robot.update_theta(2, theta3)
        robot.forward_kinematics()
        X = robot.get_all_X()
        Y = robot.get_all_Y()
        Z = robot.get_all_Z()
        points.append([X, Y, Z])
        input_pred.append([robot.get_X(), robot.get_Y(), robot.get_Z()])
        final_X.append(robot.get_X())
        final_Y.append(robot.get_Y())
        final_Z.append(robot.get_Z())
        if (robot.get_X() >= 0 and robot.get_Y() >= 0) or (robot.get_X() < 0 and robot.get_Y() < 0):
            test_set_input1.append(list(scaler_X1.transform([[robot.get_X(), robot.get_Y(), robot.get_Z()]])[0]))
            test_set_output1.append([i, theta2, theta3])
        else:
            test_set_input2.append(list(scaler_X2.transform([[robot.get_X(), robot.get_Y(), robot.get_Z()]])[0]))
            test_set_output2.append([i, theta2, theta3])
        theta2 += 1
        theta3 += 1

In [ ]:
X_2, Y_2, Z_2 = [], [], []
input_pred_2 = []
final_X_2, final_Y_2, final_Z_2 = [], [], []
points_2 = []
theta2 = 0
theta3 = -90
robot.update_theta(0, 100)
robot.update_theta(1, theta2)
robot.update_theta(2, theta3)
for i in range(-90, 90, 10):
    robot.update_theta(2, i)
    robot.forward_kinematics()
    X = robot.get_all_X()
    Y = robot.get_all_Y()
    Z = robot.get_all_Z()
    points_2.append([X, Y, Z])
    input_pred_2.append([robot.get_X(), robot.get_Y(), robot.get_Z()])
    final_X_2.append(robot.get_X())
    final_Y_2.append(robot.get_Y())
    final_Z_2.append(robot.get_Z())
    if (robot.get_X() >= 0 and robot.get_Y() >= 0) or (robot.get_X() < 0 and robot.get_Y() < 0):
        test_set_input1.append(list(scaler_X1.transform([[robot.get_X(), robot.get_Y(), robot.get_Z()]])[0]))
        test_set_output1.append([100, theta2, i])
    else:
        test_set_input2.append(list(scaler_X2.transform([[robot.get_X(), robot.get_Y(), robot.get_Z()]])[0]))
        test_set_output2.append([100, theta2, i])

In [ ]:
fig = go.Figure(data=go.Scatter3d(
    x=final_X, y=final_Y, z=final_Z,
    marker=dict(
        size=0,
        color=3,
        colorscale='Viridis',
    ),
    line=dict(
        color='darkblue',
        width=2
    )
))

fig.update_layout(scene = dict(
                                xaxis = dict(nticks=8, range=[min(final_X) - 1,max(final_X) + 1]),
                                yaxis = dict(nticks=8, range=[min(final_Y) - 1,max(final_Y) + 1]),
                                zaxis = dict(nticks=8, range=[min(final_Z) - 1,max(final_Z) + 1])),
                  width=700, height=700)
fig.show()

In [ ]:
fig = go.Figure(data=go.Scatter3d(
    x=final_X_2, y=final_Y_2, z=final_Z_2,
    marker=dict(
        size=0,
        color=3,
        colorscale='Viridis',
    ),
    line=dict(
        color='darkblue',
        width=2
    )
))

fig.update_layout(scene = dict(
                                xaxis = dict(nticks=8, range=[min(final_X_2) - 1,max(final_X_2) + 1]),
                                yaxis = dict(nticks=8, range=[min(final_Y_2) - 1,max(final_Y_2) + 1]),
                                zaxis = dict(nticks=8, range=[min(final_Z_2) - 1,max(final_Z_2) + 1])),
                  width=700, height=700)
fig.show()

### Definindo topologia da rede

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Activation, LeakyReLU, Input, Lambda, Concatenate, Dropout

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
class MAEHistory(tf.keras.callbacks.Callback):

    def __init__(self, train=None, robot=None, scaler=None, validation=None):
        super(MAEHistory, self).__init__()
        self.validation = validation
        self.train = train
        self.robot = robot
        self.scaler = scaler

    def on_epoch_end(self, epoch, logs={}):

        logs['MAE_score_train'] = float('-inf')
        X_train, y_train = self.train[0], self.train[1]
        y_pred = (self.model.predict(X_train))
        points_final = []
        for theta1, theta2, theta3 in y_pred:
            self.robot.update_theta(0, theta1)
            self.robot.update_theta(1, theta2)
            self.robot.update_theta(2, theta3)
            self.robot.forward_kinematics()
            points_final.append([robot.get_X(), robot.get_Y(), robot.get_Z()])
        score = mean_absolute_error(points_final, self.scaler.inverse_transform(X_train))      

        if (self.validation):
            logs['MAE_score_val'] = float('-inf')
            X_valid, y_valid = self.validation[0], self.validation[1]
            y_val_pred = (self.model.predict(X_valid))
            points_final = []
            for theta1, theta2, theta3 in y_val_pred:
                self.robot.update_theta(0, theta1)
                self.robot.update_theta(1, theta2)
                self.robot.update_theta(2, theta3)
                self.robot.forward_kinematics()
                points_final.append([robot.get_X(), robot.get_Y(), robot.get_Z()])
            val_score = mean_absolute_error(points_final, self.scaler.inverse_transform(X_valid))
            logs['MAE_score_train'] = np.round(score, 5)
            logs['MAE_score_val'] = np.round(val_score, 5)
            print("MAE train:", np.round(score, 5))
            print("MAE test:", np.round(val_score, 5))
        else:
            logs['MAE_score_train'] = np.round(score, 5)

In [ ]:
model1 = keras.Sequential([
    Dense(256, input_shape=(3,)),
    LeakyReLU(),
    Dropout(0.2),
    Dense(256),
    LeakyReLU(),
    Dropout(0.2), 
    Dense(256),
    LeakyReLU(),
    Dropout(0.2),
    Dense(256),
    LeakyReLU(),
    Dropout(0.2),
    Dense(256),
    LeakyReLU(),
    Dropout(0.2), 
    Dense(256),
    LeakyReLU(),
    Dropout(0.2),
    Dense(256),
    LeakyReLU(),
    Dense(3)
])

In [ ]:
model2 = keras.Sequential([
    Dense(256, input_shape=(3,)),
    LeakyReLU(),
    Dropout(0.2),
    Dense(256),
    LeakyReLU(),
    Dropout(0.2), 
    Dense(256),
    LeakyReLU(),
    Dropout(0.2),
    Dense(256),
    LeakyReLU(),
    Dropout(0.2),
    Dense(256),
    LeakyReLU(),
    Dropout(0.2), 
    Dense(256),
    LeakyReLU(),
    Dropout(0.2),
    Dense(256),
    LeakyReLU(),
    Dense(3)
])

In [ ]:
model_total = keras.Sequential([
    Dense(256, input_shape=(3,)),
    LeakyReLU(),
    Dropout(0.2),
    Dense(256),
    LeakyReLU(),
    Dropout(0.2), 
    Dense(256),
    LeakyReLU(),
    Dropout(0.2),
    Dense(256),
    LeakyReLU(),
    Dropout(0.2),
    Dense(256),
    LeakyReLU(),
    Dropout(0.2), 
    Dense(256),
    LeakyReLU(),
    Dropout(0.2),
    Dense(256),
    LeakyReLU(),
    Dense(3)
])

In [ ]:
learning_rate_fn = keras.optimizers.schedules.InverseTimeDecay(
  1e-2, 250, 0.4)

model1.compile(optimizer = keras.optimizers.Adam(learning_rate=learning_rate_fn), loss='mse')
model2.compile(optimizer = keras.optimizers.Adam(learning_rate=learning_rate_fn), loss='mse')
model_total.compile(optimizer = keras.optimizers.Adam(learning_rate=learning_rate_fn), loss='mse')

In [ ]:
model1.summary(), model2.summary(), model_total.summary()

In [ ]:
history = model1.fit(X_zone1, y_zone1, epochs=1000, batch_size=32, workers=-1, use_multiprocessing=True,
                    callbacks=[MAEHistory(train=(X_zone1, y_zone1), robot=robot, scaler=scaler_X1, validation=(test_set_input1, test_set_output1))])

In [ ]:
plt.figure(figsize=(20,20))
plt.plot(history.history['MAE_score_train'])
plt.plot(history.history['MAE_score_val'])
plt.ylabel('MAE', fontsize=30)
plt.xlabel('Epoch', fontsize=30)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.legend(['Treino', 'Teste'], loc='upper right', fontsize=25)
plt.savefig('model_zone_1.png')
plt.show()

In [ ]:
history_2 = model2.fit(X_zone2, y_zone2, epochs=1000, batch_size=32, workers=-1, use_multiprocessing=True,
                        callbacks=[MAEHistory(train=(X_zone2, y_zone2), robot=robot, scaler=scaler_X2, validation=(test_set_input2, test_set_output2))])

In [ ]:
plt.figure(figsize=(20,20))
plt.plot(history_2.history['MAE_score_train'])
plt.plot(history_2.history['MAE_score_val'])
plt.ylabel('MAE', fontsize=30)
plt.xlabel('Epoch', fontsize=30)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.legend(['Treino', 'Teste'], loc='upper right', fontsize=25)
plt.savefig('model_zone_2.png')
plt.show()

In [ ]:
history_total = model_total.fit(X_total, y_total, epochs=1000, batch_size=32, workers=-1, use_multiprocessing=True,
                    callbacks=[MAEHistory(train=(X_zone2, y_zone2), robot=robot, scaler=scaler_X2, validation=(test_set_input1 + test_set_input2, test_set_output1 + test_set_output2))])

In [ ]:
plt.figure(figsize=(20,20))
plt.plot(history_total.history['MAE_score_train'])
plt.plot(history_total.history['MAE_score_val'])
plt.ylabel('MAE', fontsize=30)
plt.xlabel('Epoch', fontsize=30)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.legend(['Treino', 'Teste'], loc='upper right', fontsize=25)
plt.savefig('model_total.png')
plt.show()

In [ ]:
model1.save("models/model1")
model2.save("models/model2")
model_total.save("models/model_total")

## Predict

### Modelo sem zonas

In [ ]:
output = []
for X, Y, Z in input_pred:
    output.append(model_total.predict(scaler_Xtotal.transform([[X, Y, Z]]))[0])

final_points = []
final_X_, final_Y_, final_Z_ = [], [], []
for theta1, theta2, theta3 in output:
    robot.update_theta(0, theta1)
    robot.update_theta(1, theta2)
    robot.update_theta(2, theta3)
    robot.forward_kinematics()
    final_X_.append(robot.get_X())
    final_Y_.append(robot.get_Y())
    final_Z_.append(robot.get_Z())
    final_points.append([robot.get_X(), robot.get_Y(), robot.get_Z()])

In [ ]:
fig = go.Figure(data=go.Scatter3d(
    x=final_X_, y=final_Y_, z=final_Z_,
    marker=dict(
        size=0,
        color=3,
        colorscale='Viridis',
    ),
    line=dict(
        color='orange',
        width=2
    ),
    name="Predito"
))

fig.add_trace(
go.Scatter3d(
    x=final_X, y=final_Y, z=final_Z,
    marker=dict(
        size=0,
        color=3,
        colorscale='Viridis',
    ),
    line=dict(
        color='darkblue',
        width=2
    ),
    name="Real"
))

fig.update_layout(scene = dict(
                                 xaxis = dict(nticks=8, range=[-110,110]),
                                yaxis = dict(nticks=8, range=[-110, 110]),
                                zaxis = dict(nticks=8, range=[0, 250])),
                  width=700, height=700)
fig.show()

In [ ]:
output = []
for X, Y, Z in input_pred_2:
    output.append(model_total.predict(scaler_Xtotal.transform([[X, Y, Z]]))[0])

points_final = []
final_X_, final_Y_, final_Z_ = [], [], []
for theta1, theta2, theta3 in output:
    robot.update_theta(0, theta1)
    robot.update_theta(1, theta2)
    robot.update_theta(2, theta3)
    robot.forward_kinematics()
    final_X_.append(robot.get_X())
    final_Y_.append(robot.get_Y())
    final_Z_.append(robot.get_Z())
    points_final.append([robot.get_X(), robot.get_Y(), robot.get_Z()])

In [ ]:
fig = go.Figure(data=go.Scatter3d(
    x=final_X_, y=final_Y_, z=final_Z_,
    marker=dict(
        size=0,
        color=3,
        colorscale='Viridis',
    ),
    line=dict(
        color='orange',
        width=2
    ),
    name="Predito"
))

fig.add_trace(
go.Scatter3d(
    x=final_X_2, y=final_Y_2, z=final_Z_2,
    marker=dict(
        size=0,
        color=3,
        colorscale='Viridis',
    ),
    line=dict(
        color='darkblue',
        width=2
    ),
    name="Real"
))
fig.update_layout(scene = dict(
                                xaxis = dict(nticks=8, range=[-110,110]),
                                yaxis = dict(nticks=8, range=[-110, 110]),
                                zaxis = dict(nticks=8, range=[0, 200])),
                   width=700, height=700)
fig.show()

### Modelo de 2 zonas

In [ ]:
output = []
for X, Y, Z in input_pred:
    if (X >= 0 and Y >= 0) or (X < 0 and Y < 0):
        output.append(model1.predict(scaler_X1.transform([[X, Y, Z]]))[0])
    else:
        output.append(model2.predict(scaler_X2.transform([[X, Y, Z]]))[0])

        
final_points = []
final_X_, final_Y_, final_Z_ = [], [], []
for theta1, theta2, theta3 in output:
    robot.update_theta(0, theta1)
    robot.update_theta(1, theta2)
    robot.update_theta(2, theta3)
    robot.forward_kinematics()
    final_X_.append(robot.get_X())
    final_Y_.append(robot.get_Y())
    final_Z_.append(robot.get_Z())
    final_points.append([robot.get_X(), robot.get_Y(), robot.get_Z()])

In [ ]:
fig = go.Figure(data=go.Scatter3d(
    x=final_X_, y=final_Y_, z=final_Z_,
    marker=dict(
        size=0,
        color=3,
        colorscale='Viridis',
    ),
    line=dict(
        color='orange',
        width=2
    ),
    name="Predito"
))

fig.add_trace(
go.Scatter3d(
    x=final_X, y=final_Y, z=final_Z,
    marker=dict(
        size=0,
        color=3,
        colorscale='Viridis',
    ),
    line=dict(
        color='darkblue',
        width=2
    ),
    name="Real"
))

fig.update_layout(scene = dict(
                                 xaxis = dict(nticks=8, range=[-110,110]),
                                yaxis = dict(nticks=8, range=[-110, 110]),
                                zaxis = dict(nticks=8, range=[0, 250])),
                  width=700, height=600)
fig.show()

In [ ]:
output = []
for X, Y, Z in input_pred_2:
    if (X >= 0 and Y >= 0) or (X < 0 and Y < 0):
        output.append(model1.predict(scaler_X1.transform([[X, Y, Z]]))[0])
    else:
        output.append(model2.predict(scaler_X2.transform([[X, Y, Z]]))[0])

points_final = []
final_X_, final_Y_, final_Z_ = [], [], []
for theta1, theta2, theta3 in output:
    robot.update_theta(0, theta1)
    robot.update_theta(1, theta2)
    robot.update_theta(2, theta3)
    robot.forward_kinematics()
    final_X_.append(robot.get_X())
    final_Y_.append(robot.get_Y())
    final_Z_.append(robot.get_Z())
    points_final.append([robot.get_X(), robot.get_Y(), robot.get_Z()])

In [ ]:
fig = go.Figure(data=go.Scatter3d(
    x=final_X_, y=final_Y_, z=final_Z_,
    marker=dict(
        size=0,
        color=3,
        colorscale='Viridis',
    ),
    line=dict(
        color='orange',
        width=2
    ),
    name="Predito"
))

fig.add_trace(
go.Scatter3d(
    x=final_X_2, y=final_Y_2, z=final_Z_2,
    marker=dict(
        size=0,
        color=3,
        colorscale='Viridis',
    ),
    line=dict(
        color='darkblue',
        width=2
    ),
    name="Real"
))
fig.update_layout(scene = dict(
                                xaxis = dict(nticks=8, range=[-110,110]),
                                yaxis = dict(nticks=8, range=[-110, 110]),
                                zaxis = dict(nticks=8, range=[0, 200])),
                   width=700, height=700)
fig.show()